In [1]:
import numpy as np

navs = ('No navegable', 'Navegable')

x = np.load('processed_nav_data.npy')
y = np.load('nav_data_labels.npy')

n_clases = len(navs)
n_features = len(x[0])

print(np.unique(y, return_counts=True))
print(len(x))
print(len(y))
print(n_clases)
print(n_features)


(array([0, 1]), array([325, 524]))
849
849
2
10


In [2]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(y),
                                                  y)
kf = KFold(n_splits=5, shuffle=True)
acc = 0
recall = np.array([0., 0.])
iteration = 1
for train_index, test_index in kf.split(x):
    print(f'Iteration {iteration}')
    iteration += 1
    x_train = x[train_index, :]
    y_train = y[train_index]

    clf = Sequential()
    clf.add(Dense(16, input_dim=n_features, activation='relu'))
    clf.add(Dense(16, activation='relu'))
    clf.add(Dense(1, activation='sigmoid'))
    clf.compile(loss='binary_crossentropy', optimizer='adam')
    history = clf.fit(x_train, y_train, epochs=50, batch_size=5, class_weight=(
        dict(zip(np.unique(y), class_weights))), verbose=0)

    x_test = x[test_index, :]
    y_test = y[test_index]
    y_pred = (clf.predict(x_test) > 0.5).astype("int32")
    cm = confusion_matrix(y_test, y_pred)
    acc += (cm[0, 0]+cm[1, 1])/len(y_test)
    recall[0] += cm[0, 0]/(cm[0, 0] + cm[0, 1])
    recall[1] += cm[1, 1]/(cm[1, 0] + cm[1, 1])

acc = acc/5
print('ACC = ', acc)
recall = recall/5
print('RECALL = ', recall)


2021-11-15 13:53:08.215792: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/hivini/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[0 1], y=[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 0 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 0 0 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1
 1 1 1 1 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 1 0 1
 0 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1
 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 

Iteration 1


2021-11-15 13:53:09.373209: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-15 13:53:09.380729: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-15 13:53:09.652739: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-15 13:53:09.652821: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 with Max-Q Design computeCapability: 7.5
coreClock: 1.215GHz coreCount: 46 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 357.69GiB/s
2021-11-15 13:53:09.652873: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-11-15 13:53:09.654337: I tensorflow/stre

Iteration 2
Iteration 3
Iteration 4
Iteration 5
ACC =  0.9316811695092237
RECALL =  [0.89635962 0.95459959]
